In [1]:
class Card:
    def __init__(self, value, suit, visible=False, bonus=False):
        """
        Initialize a card with given value, suit, visibility, and bonus status.
        
        Parameters:
        - value (int): The rank of the card (1 to 13, where 1 = Ace and 13 = King).
        - suit (int): The suit of the card (1 to 4).
        - visible (bool): Whether the card is face-up.
        - bonus (bool): Whether this card has been given a bonus for moving to the foundation.
        """
        self.value = value  # 1 to 13
        self.suit = suit  # 1 to 4, even suits are red, odd are black
        # 1 - ♣ (clubs)
        # 2 - ♦ (diamonds)
        # 3 - ♠ (spades)
        # 4 - ♥ (hearts) 
        self.visible = visible  # Face-up or face-down
        self.bonus = bonus  # Bonus flag to prevent duplicate rewards

    def __repr__(self):
        visibility = "Visible" if self.visible else "Hidden"
        return f"Card(value={self.value}, suit={self.suit}, {visibility}, bonus={self.bonus})"


In [7]:
%pip install gym

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
%pip install rich

[notice] A new release of pip is available: 23.0.1 -> 24.3.1


[notice] To update, run: python.exe -m pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: mdurl~=0.1 in c:\users\gachi\appdata\local\programs\python\python310\lib\site-packages (from markdown-it-py>=2.2.0->rich) (0.1.2)



In [2]:
from rich.console import Console

# Create a Console object
console = Console()

# Print colored text
console.print("This is red text", style="bold red")
console.print("This is green text", style="black")
console.print("This is blue text on yellow background", style="bold blue on yellow")


This is red text

This is green text

This is blue text on yellow background

In [ ]:
import gym
from gym import spaces
import random
from rich.console import Console
from rich.text import Text

# Initialize a Console object from the rich library for printing with styles
console = Console()

class SolitaireEnv(gym.Env):
    def __init__(self):
        super(SolitaireEnv, self).__init__()
        # The action space now includes three parts: action type, source column, destination column
        self.action_space = spaces.Tuple((
            spaces.Discrete(3),    # Action type: 0 (Move within tableau), 1 (Draw card), 2 (Move to foundation)
            spaces.Discrete(7),    # Source column (0-6)
            spaces.Discrete(7),    # Destination column (0-6)
            spaces.Discrete(7),     # Card index (0-6, only used for tableau moves)
            spaces.Discrete(4)
        ))
        # Define observation space with structured tableau, foundation, and draw pile
        self.observation_space = spaces.Dict({
            'tableau': spaces.MultiDiscrete([53] * 7 * 7),  # 7 columns, each with 7 slots
            'foundation': spaces.MultiDiscrete([14, 14, 14, 14]),  # Four foundation piles
            'draw_pile': spaces.Discrete(53)  # One card drawn at a time
        })

        # Track revealed cards from the draw pile
        self.revealed_cards = []
        self.draw_index = 0  # Tracks current position in draw pile

        self.tableau = None
        self.foundation = None
        self.draw_pile = None
        self.done = False
        self._reset_game_state()

    def _reset_game_state(self):
        # Initialize the deck as a list of Card objects
        deck = [Card(value, suit) for suit in range(1, 5) for value in range(1, 14)]
        random.shuffle(deck)

        # Initialize tableau with some cards face-down
        self.tableau = [[deck.pop() for _ in range(i + 1)] for i in range(7)]
        for col in self.tableau:
            for card in col[:-1]:
                card.visible = False  # Face-down
            col[-1].visible = True  # Top card face-up

        # Foundation starts empty
        self.foundation = [[] for _ in range(4)]
        self.draw_pile = deck  # Remaining cards in the draw pile
        self.done = False

    def reset(self):
        """Resets the environment to the initial state."""
        self._reset_game_state()
        return self._get_observation()

    def _get_observation(self):
        # This function will need to convert the complex Card objects to a simple format for Gym
        tableau_observation = [
            [card.value if card.visible else 0 for card in column] for column in self.tableau
        ]
        foundation_observation = [len(pile) for pile in self.foundation]
        draw_pile_observation = self.draw_pile[0].value if self.draw_pile else 0

        return {
            'tableau': tableau_observation,
            'foundation': foundation_observation,
            'draw_pile': draw_pile_observation
        }

    def step(self, action: list[int]):
        action_type, source, destination, card_index = action
        reward = -1  # Base penalty for each action

        if action_type == 0:  # Move Card within Tableau
            valid_move_made = self._move_within_tableau(source, destination, card_index)
            if valid_move_made:
                reward += 0
            else:
                reward -= 1  # Extra penalty for invalid move

        elif action_type == 1:  # Draw Card from Draw Pile
            if self.draw_pile:
                drawn_card = self.draw_pile.pop(0)
                drawn_card.visible = True  # Drawn card should be face-up
                self.tableau[0].append(drawn_card)  # Place the card on the first tableau column

        elif action_type == 2:  # Move Card to Foundation
            foundation_move_successful = self._move_to_foundation(source)
            if foundation_move_successful:
                reward += 130

        flipped_count = self._flip_visible_cards()
        reward += flipped_count * 20

        # Check if game is won (all foundations complete)
        if all(len(foundation) == 13 for foundation in self.foundation):
            self.done = True

        return self._get_observation(), reward, self.done, {}

    def _draw_card(self):
        # Reveal up to 3 cards at a time from the draw pile
        
        if self.draw_index >= len(self.draw_pile):
            self.draw_index = 0  # Restart the draw pile if we reach the end
        if self.draw_pile:
            card = self.draw_pile[self.draw_index]
            card.visible = True
            self.revealed_cards.append(card)
            self.draw_index += 1


    def _move_within_tableau(self, source, destination, card_index=None):
        # If the source is from the draw pile (denoted by a special index, say -1)
        if source == -1:
            if not self.revealed_cards:
                return False  # No cards revealed in the draw pile
            # Use the last revealed card from the draw pile
            card_to_move = self.revealed_cards[-1]
            
            # Check if destination column is empty (only Kings can move to empty columns)
            if not self.tableau[destination]:
                if card_to_move.value == 13:
                    self.tableau[destination].append(card_to_move)
                    self.revealed_cards.pop()  # Remove from revealed list
                    return True
                else:
                    return False  # Only Kings can move to an empty column

            # Check if the move is valid based on the destination column's top card
            dest_card = self.tableau[destination][-1]
            if (card_to_move.value == dest_card.value - 1 and
                    (card_to_move.suit % 2) != (dest_card.suit % 2)):
                self.tableau[destination].append(card_to_move)
                self.revealed_cards.pop()  # Remove from revealed list
                return True

            return False  # Invalid move for draw pile card
        
        # Validate source and destination columns
        if source < 0 or source >= len(self.tableau) or destination < 0 or destination >= len(self.tableau):
            return False  # Invalid source or destination
        
        # Check if the source column is non-empty and contains enough cards for the card index
        if not self.tableau[source] or (card_index is not None and card_index >= len(self.tableau[source])):
            return False
        
        # If no card_index is specified, assume we're moving the top card
        if card_index is None:
            card_index = len(self.tableau[source]) - 1

        cards_to_move = self.tableau[source][card_index:]

        # Check if destination column is empty (only Kings can be moved to an empty column)
        if not self.tableau[destination]:
            if cards_to_move[0].value == 13:
                # Move the sequence
                self.tableau[destination].extend(cards_to_move)
                del self.tableau[source][card_index:]
                return True
            else:
                return False  # Only Kings can be moved to an empty column

        # Check if the move is valid based on the destination column’s top card
        dest_card = self.tableau[destination][-1]
        if (cards_to_move[0].value == dest_card.value - 1 and
                (cards_to_move[0].suit % 2) != (dest_card.suit % 2)):
            # Move the sequence
            self.tableau[destination].extend(cards_to_move)
            del self.tableau[source][card_index:]
            return True
        
        return False  # Move was invalid
    

    def _move_to_foundation(self, source):
        # If source is the draw pile (denoted by -1), take the last revealed card
        if source == -1:
            if not self.revealed_cards:
                return False  # No revealed cards in draw pile
            card = self.revealed_cards[-1]
            foundation_index = card.suit - 1  # Determine foundation based on suit

            if len(self.foundation[foundation_index]) == card.value - 1:
                # Move the card to the foundation and remove from revealed list
                self.foundation[foundation_index].append(card)
                self.revealed_cards.pop()
                return True

            return False  # Invalid move
        
        # Validate source column
        if source < 0 or source >= len(self.tableau) or not self.tableau[source]:
            return False  # Invalid move, no card to move
        
        # Get the top card from the source column
        card = self.tableau[source][-1]
        foundation_index = card.suit - 1  # Determine foundation pile based on suit
        
        # Check if the card can move to the foundation (must be in ascending order)
        if len(self.foundation[foundation_index]) == card.value - 1:
            # Move the card to the foundation and remove from tableau
            self.foundation[foundation_index].append(self.tableau[source].pop())
            return True
        
        return False  # Move was invalid
    

    def _flip_visible_cards(self):
        flipped_count = 0
        for column in self.tableau:
            if column and not column[-1].visible:  # If the top card is face-down
                column[-1].visible = True  # Flip it face-up
                flipped_count += 1
        return flipped_count




    def render(self, mode='human'):
        # Foundations
        foundation_str = []
        for pile in self.foundation:
            if pile:
                card = pile[-1]
                suit = ['♣', '♦', '♠', '♥'][card.suit - 1]
                # Apply color red for red suits (Diamonds and Hearts)
                if card.suit in [2, 4]:  # Red suits: Diamonds (2) and Hearts (4)
                    suit = Text(suit, style="red")  # Red color for red suits
                foundation_str.append(f"| {card.value} {suit} |")
            else:
                foundation_str.append("|     |")  # Empty foundation pile

        # Print foundation row
        console.print("Foundations:", "  ".join(foundation_str))

        # Tableau - display all 7 columns in a single row
        tableau_str = []
        for col in self.tableau:
            tableau_str.append(" ".join([f"┌─────┐" if card.visible else "┌─────┐" for card in col]))  # Card tops
            tableau_str.append(" ".join([f"| {card.value if card.visible else ' ?'}{' ' if card.visible and len(str(card.value)) != 2 else ''}{[' ♣', '[bold red] ♦[/bold red]', ' ♠', '[bold red] ♥[/bold red]'][card.suit - 1] if card.visible else '  '}|" for card in col]))  # Card values
            tableau_str.append(" ".join([f"|     |" for _ in col]))  # Empty space for spacing between cards
            tableau_str.append(" ".join([f"└─────┘" for _ in col]))  # Card bottoms

        # Print tableau columns in one row
        tableau_str = '\n'.join(tableau_str)
        console.print("Tableau:\n" + tableau_str)

        # Draw pile (remaining count, last 3 revealed, discarded count)
        draw_pile_display = f"Draw Pile: {len(self.draw_pile)} cards remaining"
        
        # Last 3 revealed cards (if any)
        last_three = [str(card.value) if card.visible else '?' for card in self.draw_pile[:3]]
        last_three_display = f"\nLast 3 Drawn: {' '.join(last_three)}"
        
        # Discarded cards (number of discarded cards)

        # Print the full draw pile row
        console.print(draw_pile_display, last_three_display)






In [6]:
env = SolitaireEnv()

# Reset the environment to get the initial state
env.reset()

# Render the initial state
env.render()

Foundations: |     |  |     |  |     |  |     |

Tableau:
┌─────┐
| 13 ♥|
|     |
└─────┘
┌─────┐ ┌─────┐
|  ?  | | 13 ♣|
|     | |     |
└─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | | 4  ♣|
|     | |     | |     |
└─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | | 2  ♦|
|     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | | 12 ♠|
|     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | |  ?  | | 5  ♥|
|     | |     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | |  ?  | |  ?  | | 8  ♣|
|     | |     | |     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘

Draw Pile: 24 cards remaining 
Last 3 Drawn: ? ? ?

In [131]:
from rich.console import Console
from rich.text import Text

console = Console()

# Create text with different styles
text_part1 = Text("Hello ", style="bold green")  # Bold green
text_part2 = Text("World", style="bold red")    # Bold red

# Print them in a single call
console.print(text_part1 + text_part2)


Hello World